In [6]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'                
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'              


In [7]:
!pip install tensorflow

In [8]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [9]:
input_texts = ['hi', 'how are you?', 'what is your name?', 'bye']
target_texts = ['\thello\n', '\ti am fine\n', '\ti am a chatbot\n', '\tgoodbye\n']

In [10]:
input_characters = sorted(set(''.join(input_texts)))
target_characters = sorted(set(''.join(''.join(target_texts))))

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

max_encoder_seq_length = max(len(txt) for txt in input_texts)
max_decoder_seq_length = max(len(txt) for txt in target_texts)

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [11]:
latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=16, epochs=300)

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 1.7628
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 1.7494
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 1.7383
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.7277
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 1.7167
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.7041
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 1.6881
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 1.6638
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step - loss: 1.6215
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - loss: 1.5833
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.5688
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.5555
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 1.5553
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 1.5150
Epoch 15/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 1.4989
Epoch 16/300
1/1 ━━

In [12]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [13]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence.strip()

In [ ]:
def chatbot_response(text):
    input_seq = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, char in enumerate(text):
        if char in input_token_index:
            input_seq[0, t, input_token_index[char]] = 1.
    return decode_sequence(input_seq)

while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Bot: Goodbye!")
        break
    print("Bot:", chatbot_response(user_input))

You:  hii


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Bot: hello


You:  bye


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Bot: goodbye


You:  what is your name


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Bot: i am a aaaa
